In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/comp/comp3/company_i

/content/drive/MyDrive/comp/comp3/company_i


In [3]:
!pip install japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=2992c3bfecd82763983fb980d780b22f6d5b3fcc5aea9882e48fdaea2ca6023d
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [4]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.2 MB/s eta 0:00:00


In [163]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from category_encoders import TargetEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import japanize_matplotlib as plt
%matplotlib inline

# 小数第３位まで表示
%precision 3

'%.3f'

In [164]:
# セルの横幅を広げる
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [222]:
data = pd.read_csv('data.csv')

data = data.drop(['Over18',"StandardHours","EmployeeCount"], axis=1)

data.replace('No',0, inplace=True)
data.replace('Yes',1, inplace=True)
data.replace('Male',0, inplace=True)
data.replace('Female',1, inplace=True)

In [223]:
le = LabelEncoder()

encode_columns= ['BusinessTravel','Department','EducationField','MaritalStatus','HowToEmploy']
for column in encode_columns:
  data[column] = le.fit_transform(data[column])
# 0,1,2,6
data.replace('Sales Executive',0,inplace=True)
data.replace('Research Scientist',1,inplace=True)
data.replace('Laboratory Technician',2,inplace=True)
data.replace('Manufacturing Director',3,inplace=True)
data.replace('Healthcare Representative',4,inplace=True)
data.replace('Manager',5,inplace=True)
data.replace('Sales Representative',6,inplace=True)
data.replace('Research Director',7,inplace=True)
data.replace('Human Resources',8,inplace=True)

In [224]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
import numpy as np

target = data['Attrition']
explain = data.drop(['Attrition'], axis=1)

# データをロード
trainingx, validx, trainingy, validy = train_test_split(explain, target, test_size=0.2, shuffle=True)


trainx, testx, trainy, testy = train_test_split(trainingx, trainingy, test_size=0.2, random_state=1)

# モデルの定義
rf = LGBMClassifier(importance_type='gain')

# クロスバリデーションの設定
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 評価指標の設定
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro')
}

# クロスバリデーションでモデルのパフォーマンスを評価
cv_results = cross_validate(rf, trainx, trainy, cv=cv, scoring=scoring, return_train_score=True)

# 各スコアを表示
print(f"Cross-validation accuracy scores: {cv_results['test_accuracy']}")
print(f"Mean cross-validation accuracy: {np.mean(cv_results['test_accuracy'])}")

print(f"Cross-validation precision scores: {cv_results['test_precision']}")
print(f"Mean cross-validation precision: {np.mean(cv_results['test_precision'])}")

print(f"Cross-validation recall scores: {cv_results['test_recall']}")
print(f"Mean cross-validation recall: {np.mean(cv_results['test_recall'])}")

# クロスバリデーションで特徴重要度を計算する関数
def cross_val_feature_importance(model, X, y, cv):
    feature_importances = np.zeros(X.shape[1])

    for train_idx, val_idx in cv.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        feature_importances += model.feature_importances_

    return feature_importances / cv.get_n_splits()

# クロスバリデーションで特徴重要度を計算
feature_importances = cross_val_feature_importance(rf, trainx, trainy, cv)

# テストセットでのスコアを計算
rf.fit(trainx, trainy)
test_accuracy = accuracy_score(testy, rf.predict(testx))
test_precision = precision_score(testy, rf.predict(testx), average='macro')
test_recall = recall_score(testy, rf.predict(testx), average='macro')

print(f'Test accuracy: {test_accuracy}')
print(f'Test precision: {test_precision}')
print(f'Test recall: {test_recall}')

# 特徴重要度を表示
print('Feature importances:', feature_importances)

[LightGBM] [Info] Number of positive: 132, number of negative: 620
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000260 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1547
[LightGBM] [Info] Number of data points in the train set: 752, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.175532 -> initscore=-1.546918
[LightGBM] [Info] Start training from score -1.546918
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [225]:
print(rf.score(validx, validy))
print(accuracy_score(validy, rf.predict(validx)) )
print(precision_score(validy, rf.predict(validx), average='macro'))
print(recall_score(validy, rf.predict(validx), average='macro'))

0.8877551020408163
0.8877551020408163
0.7945524226663017
0.610608552631579


In [226]:
predictions = rf.predict(validx)
predictions_series = pd.Series(predictions)  # Convert predictions to a Pandas Series
print(predictions_series.value_counts())

0    281
1     13
Name: count, dtype: int64


In [229]:
validx.loc[
    (validx['PerformanceRating'] == 1) &
    (validx['Incentive'] == 0) &
    (validx['JobRole'].isin([0, 1, 2, 6])),
    'Incentive'
] = 100

In [ ]:
# validx.loc[
#     (validx['Incentive'] == 1),
#     'PerformanceRating'
# ] = 2

In [230]:
validx['PerformanceRating'].value_counts()

PerformanceRating
3    101
2     88
1     53
4     52
Name: count, dtype: int64

In [231]:
predictions = rf.predict(validx)
predictions_series = pd.Series(predictions)  # Convert predictions to a Pandas Series
print(predictions_series.value_counts())

0    279
1     15
Name: count, dtype: int64


In [ ]:
data['Age_bin'] = pd.cut(data['Age'], bins=range(15, 70, 5), labels=[f"{i}-{i+4}" for i in range(15, 65, 5)])

In [41]:
job_lab = data[data['JobRole']==2]

In [43]:
job_lab['IncentiveFlg'] = job_lab['Incentive'].apply(lambda x: 1 if x >= 1 else 0)

In [44]:
job_lab['IncentiveFlg'].value_counts()

IncentiveFlg
1    207
0     52
Name: count, dtype: int64

In [46]:
job_lab[['IncentiveFlg','Incentive']]

,IncentiveFlg,Incentive
4,0,0
13,0,0
16,0,0
19,0,0
22,0,0
...,...,...
1420,1,1371
1424,1,1798
1439,1,1814
1458,1,1063
